In [1]:
import time
import datetime
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pybullet as p
import pybullet_data

import torch
from torch_cluster import radius_graph, radius
from torch_scatter import scatter

import matplotlib
import matplotlib.pyplot as plt

import edf
from edf.pybullet_env.utils import get_image, axiscreator, img_data_to_pointcloud
from edf.visual_utils import plot_color_and_depth, scatter_plot, scatter_plot_ax
from edf.pybullet_env.env import MugTask

device = 'cpu'
gpu_idx = 0
torch.cuda.set_device(gpu_idx)

pybullet build time: Dec  1 2021 18:34:28
/home/hw/anaconda3/envs/SE3_Transporter/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

stride = (7, 7)
stride_pick = (10, 10)
stride_place = (7, 7)

folder_name = 'demo'
task_name = 'mug_task'
path = f'{folder_name}/{task_name}'
task = MugTask(use_gui=False)

upright_pose_seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
lying_pose_seeds= [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

upright_samples = []
for seed in upright_pose_seeds:
    task.reset(seed = seed, cup_pose='upright')
    pc = task.observe_pointcloud(stride=stride)
    pre_grasp, grasp = task.oracle_pick_handle(random_180_flip=False)
    sample = {}
    sample['coord'] = pc['coord']
    sample['color'] = pc['color']
    sample['range'] = pc['ranges']
    sample['pre_grasp'] = pre_grasp 
    sample['grasp'] = grasp
    sample['images'] = task.observe()
    sample['center'] = task.center
    sample['stride'] = stride

    task.pick(pre_grasp, grasp, sleep=False)
    task.retract_robot(gripper_val=1.)
    pc = task.observe_pointcloud_pick(stride=stride_pick)
    sample['coord_pick'] = pc['coord']
    sample['color_pick'] = pc['color']
    sample['range_pick'] = pc['ranges']
    sample['pick_pose'] = (pc['X_sg'], pc['R_sg'])
    sample['images_pick'] = task.observe_pick()
    sample['stride_pick'] = stride_pick
    
    pc = task.observe_pointcloud(stride=stride_place)
    pre_grasp, grasp = task.oracle_place_handle()
    sample['coord_place'] = pc['coord']
    sample['color_place'] = pc['color']
    sample['range_place'] = pc['ranges']
    sample['pre_place'] = pre_grasp 
    sample['place'] = grasp
    sample['images_place'] = task.observe()
    sample['stride_place'] = stride_place

    upright_samples.append(sample)

lying_samples = []
for seed in lying_pose_seeds:
    task.reset(seed = seed, cup_pose='lying')
    pc = task.observe_pointcloud(stride=stride)
    pre_grasp, grasp = task.oracle_pick_handle(random_180_flip=False)
    sample = {}
    sample['coord'] = pc['coord']
    sample['color'] = pc['color']
    sample['range'] = pc['ranges']
    sample['pre_grasp'] = pre_grasp 
    sample['grasp'] = grasp
    sample['images'] = task.observe()
    sample['center'] = task.center
    sample['stride'] = stride

    task.pick(pre_grasp, grasp, sleep=False)
    task.retract_robot(gripper_val=1.)
    pc = task.observe_pointcloud_pick(stride=stride_pick)
    sample['coord_pick'] = pc['coord']
    sample['color_pick'] = pc['color']
    sample['range_pick'] = pc['ranges']
    sample['pick_pose'] = (pc['X_sg'], pc['R_sg'])
    sample['images_pick'] = task.observe_pick()
    sample['stride_pick'] = stride_pick
    
    pc = task.observe_pointcloud(stride=stride_place)
    pre_grasp, grasp = task.oracle_place_handle()
    sample['coord_place'] = pc['coord']
    sample['color_place'] = pc['color']
    sample['range_place'] = pc['ranges']
    sample['pre_place'] = pre_grasp 
    sample['place'] = grasp
    sample['images_place'] = task.observe()
    sample['stride_place'] = stride_place

    lying_samples.append(sample)

task.close()

import os
import gzip
import pickle

if not os.path.exists(path):
    os.makedirs(path)

with gzip.open(path + f'/upright.gzip', 'wb') as f:
    pickle.dump(upright_samples, f)

with gzip.open(path + f'/lying.gzip', 'wb') as f:
    pickle.dump(lying_samples, f)

del upright_samples, lying_samples

Identified 6433 IK solutions with minimum distance of 2.664 in 0.934 seconds
Identified 8054 IK solutions with minimum distance of 2.338 in 1.029 seconds
Identified 9133 IK solutions with minimum distance of 2.210 in 1.052 seconds
Identified 6309 IK solutions with minimum distance of 2.664 in 0.858 seconds
Identified 6670 IK solutions with minimum distance of 1.891 in 0.820 seconds
Identified 7269 IK solutions with minimum distance of 1.077 in 0.505 seconds
Identified 5736 IK solutions with minimum distance of 1.374 in 0.502 seconds
Identified 8686 IK solutions with minimum distance of 2.142 in 0.995 seconds
Identified 4930 IK solutions with minimum distance of 1.306 in 0.353 seconds
Identified 9986 IK solutions with minimum distance of 2.447 in 1.243 seconds
Identified 5091 IK solutions with minimum distance of 1.228 in 0.338 seconds
Identified 8985 IK solutions with minimum distance of 1.239 in 0.753 seconds
Identified 2244 IK solutions with minimum distance of 2.099 in 0.395 seconds